In [36]:
import os
import mido
from mido import MidiFile, MidiTrack, Message, merge_tracks
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras import utils, Input
from keras_self_attention import SeqSelfAttention
from sklearn.model_selection import train_test_split

In [37]:
#specify the path
path='music/classical-piano-type0/beethoven'


notes = []
for filename in os.listdir(path):
    if filename.endswith(".mid"):
        print(filename)
        mid = MidiFile(path + '/' + filename)
        for msg in mid:
            if not msg.is_meta and msg.channel == 0 and  msg.type == 'note_on':
                data = msg.bytes()
                notes.append(data[1])

beethoven_hammerklavier_2.mid
beethoven_les_adieux_2.mid
beethoven_opus10_2.mid
beethoven_opus22_2.mid
beethoven_opus22_3.mid


In [38]:
notes_set = np.unique(list(notes))
total_notes = len(notes_set)
print(notes_set)

[ 28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  91  92  93  94  95  96  98 100 101]


In [39]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 35

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    network_output = utils.to_categorical(network_output)

    return (network_input, network_output)

In [40]:
X,y = prepare_sequences(notes,total_notes)

In [41]:

model = Sequential()

model.add(Bidirectional(LSTM(512, input_shape=(X.shape[1:]), return_sequences=True)))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(Dropout(0.3))
model.add(LSTM(512, input_shape=(X.shape[1:]),  return_sequences=True))
model.add(Dropout(0.3))
model.add(GlobalMaxPool1D())
model.add(Dense(total_notes,activation='softmax')) 
optimizer = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
filepath="./checkpoint/checkpoint_model_ff7{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='loss', 
                                      verbose=1, save_best_only=False, 
                                      mode='min', save_freq=500)

In [ ]:
model.fit(X, y,
                 batch_size=64,
                 shuffle=True,
                 epochs=10,
                 callbacks=[model_save_callback],
                 verbose=1)
#fit(X, y, 20, 1, verbose=1, callbacks=[model_save_callback])

Epoch 1/10
175/265 [==================>...........] - ETA: 1:36 - loss: 3.4529
Epoch 00001: saving model to ./checkpoint\checkpoint_model_ff701.hdf5
265/265 [==============================] - 284s 1s/step - loss: 3.4473
Epoch 2/10
265/265 [==============================] - 288s 1s/step - loss: 3.4141
Epoch 3/10
145/265 [===============>..............] - ETA: 2:07 - loss: 3.3758
Epoch 00003: saving model to ./checkpoint\checkpoint_model_ff703.hdf5
176/265 [==================>...........] - ETA: 1:35 - loss: 3.3803

In [67]:
rand = np.random.randint(0, len(X)-1)
prediction  = []

for i in range(10):
    pred = np.reshape(X[rand], (1, len(X[rand]), 1))
    #normalize
    pred = pred / float(total_notes)
    pred_res = model.predict(pred)
    index = np.argmax(pred_res)
    prediction.append(index)

In [77]:
import random
ind = np.random.randint(0,len(X)-1)

random_music = X[ind]

predictions=[]
for i in range(500):

    random_music = random_music.reshape(1,len(random_music),1)

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [80]:
track = MidiTrack()
t = 0

for note in predictions:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 110])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)

newFile = MidiFile()
newFile.tracks.append(track)
newFile.save('beethoven_epoch5_softmax.mid')

In [ ]:
#model.save('green_model.h5')